In [2]:
import imageio
import matplotlib.pyplot as plt
from matplotlib.colors import CenteredNorm, TwoSlopeNorm
from pathlib import Path
from tqdm import tqdm
from glob import glob
import numpy as np
import xarray as xr
import pandas as pd
import datetime as dt
import seaborn as sns
import yaml

from convml_tt.data.dataset import TRIPLET_TILE_FILENAME_FORMAT

import convml_tt.interpretation.plots

import convml_tt.interpretation.embedding_transforms

from convml_tt.interpretation.plots import isomap2d

from convml_tt.data.transforms import get_transforms
from convml_tt.interpretation.embedding_transforms import apply_transform

In [1]:
#folder = "AquaHkmLabSea2022b"
folder = "Aqua1kmLabSea2022"

filepath = "/home/eefjg/OneDrive/Leeds/PhD/Data/MODIS/"+folder

filename = "embs.nc"
#filename = "embs_no_land.nc"
#filename = "embs_no_ice.nc" # This data also has no land
#filename = "embs_cao.nc"

In [3]:
tiles_ds = xr.open_dataset(filepath+"/"+filename)
tiles_ds

<xarray.Dataset>
Dimensions:        (tile_id: 10000, tile_type: 3, emb_dim: 100)
Coordinates:
  * tile_id        (tile_id) int64 0 1 2 3 4 5 ... 9994 9995 9996 9997 9998 9999
  * tile_type      (tile_type) object 'anchor' 'neighbor' 'distant'
Dimensions without coordinates: emb_dim
Data variables:
    embs           (tile_type, tile_id, emb_dim) float32 ...
    isomap_x       (tile_type, tile_id) float32 ...
    isomap_y       (tile_type, tile_id) float32 ...
    c_lat          (tile_type, tile_id) float32 ...
    c_lon          (tile_type, tile_id) float32 ...
    time           (tile_type, tile_id) datetime64[ns] ...
    land_fraction  (tile_type, tile_id) float32 ...